<img src="./img/header.png">
# Globus SDK
https://github.com/globus/globus-sdk-python


# Globus SDK Docs
http://globus.github.io/globus-sdk-python/


# Globus CLI
https://github.com/globus/globus-cli

# Requirements
- You need to be in the tutorial users group for sharing
- Installed Globus Python SDK

In [ ]:
from __future__ import print_function # for python 2
from globus_sdk import TransferClient

tutorial_endpoint_1 = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"
tutorial_endpoint_2 = "ddb59af0-6d04-11e5-ba46-22000b92c6ec"

tutorial_users_group = "50b6a29c-63ac-11e4-8062-22000ab68755"

# Configuration
First you will need to configure the client with your access credentials. You can obtain OAuth2 access tokens via the tokens.globus.org website. Click the "Jupyter Notebook" option and copy the resutling text below. 

In [ ]:
auth_token = ''
transfer_token = ''

# Transfer Client

The Globus Python SDK makes transfer functionality avaialable via a TransferClient. 

## Instantiate Transfer Client

The TransferClient can be instantiated by passing in a token or by using the token configured in your home directory (~/.globus.cfg). Tokens can be obtained via the tokens.globus.org site. 

In [ ]:
tc = TransferClient(token=transfer_token) 

## Using the client

The transfer client makes REST resources avaialble via easy to use methods. 

The response from these methods wraps the HTTP response status, content type, text and JSON response body. 

In [ ]:
endpoint = tc.get_endpoint(tutorial_endpoint_1)
print("HTTP Status: %s" % endpoint.http_status)
print("Content Type: %s" % endpoint.content_type)
print("Data: %s" % endpoint.data) # raw response text can be obtained via endpoint.text_body or endpoint.json_body

## Handling errors

If an error occurs the API will throw an error.

In [ ]:
try:
    endpoint = tc.get_endpoint("BAD ENDPOINT ID")
except Exception as ex:
    print("HTTP Status: %s" % ex[0])
    print("HTTP Error: %s" % ex[1])
    print("HTTP Message: %s" % ex[2])

# Endpoint management

## Endpoint search by name

Globus has over 8000 registered endpoints. To find endpoints of interest you can access powerful search capabilities via the SDK. For example, to search for a given string in a name: 

In [ ]:
search_str = "Globus Tutorial Endpoint"
endpoints = tc.endpoint_search(search_str)
print("==== Displaying endpoint matches for search: '%s' ==="%search_str)
for r in endpoints:
    print("%s (%s)"%(r.data["display_name"], r.data["id"]))

## Restricting search scope with filters

There are also a number of default filters to restrict the search for 'my-endpoints', 'my-gcp-endpoints',     'recently-used', 'in-use', 'shared-by-me','shared-with-me') 

In [ ]:
endpoints = tc.endpoint_search(
        filter_fulltext=search_str, filter_scope="recently-used")
for r in endpoints:
    print("%s (%s)"%(r.data["display_name"], r.data["id"]))

## Endpoint details

You can also retrieve complete information about an endpoing, including name, owner, location, and server configurations. 

In [ ]:
endpoint = tc.get_endpoint(tutorial_endpoint_1)
print("Display name: %s" % endpoint.data["display_name"])
print("Owner: %s" % endpoint.data["owner_string"])
print("ID: %s" % endpoint.data["id"])

# File operations

## Path encoding & UTF gotchas

## Autoactivate Endpoints
Globus endpoints must be "activated" before they can be used. Many endpoints (e.g., Globus Connect Personal and Shared endpoints) can be "autoactivated" using your active Globus identities.For endpoints that require activatino (e.g., MyProxy and MyProxy OAuth) you can activate those endpoints via the Globus website. Here we autoactivate the Globus tutorial endpoints using their endpoint_ids

In [ ]:
#help(tc.endpoint_activate)
ep1 = tc.endpoint_autoactivate(tutorial_endpoint_1)
ep2 = tc.endpoint_autoactivate(tutorial_endpoint_2)
print (ep1.data["message"])

In [ ]:
# retrieve activation requirements
#activation_ep = tc.endpoint_get_activation_requirements(myproxy_endpoint)
#requirements = activation_ep.data['DATA']

# complete the requirements document with credentials
#for r in requirements: 
#    if r["name"] == "username":
#        r["value"] = myproxy_username
#    elif r["name"] == "passphrase":
#        r["value"] = myproxy_password

# activate the endpoint using the completed requirements
#activated_ep = tc.endpoint_activate(myproxy_endpoint, requirements)
#print(activated_ep)

## Get a directory listing

Having activated an endpoint you can now perform operations on it. For example, performing an ls command to retrieve directory contents. 

In [ ]:
# help(tc.operation_ls)
endpoint_id = tutorial_endpoint_1
endpoint_path = '/share/godata/'
r = tc.operation_ls(endpoint_id,path=endpoint_path)
print("==== Endpoint_ls for endpoint %s %s ===="%(endpoint_id,endpoint_path))
for item in r.data['DATA']:
    print("%s: %s [%s]"%(item['type'],item['name'],item['size']))

## Make directory

In [ ]:
#help(tc.operation_mkdir)
endpoint_id = tutorial_endpoint_1
endpoint_path = '/~/tutorial_dir'
r = tc.operation_mkdir(endpoint_id,path=endpoint_path)
print(r.data["message"])

## Rename

You can rename files and directories on your endpoints. 

In [ ]:
#help(tc.operation_rename)
endpoint_id = tutorial_endpoint_1
r = tc.operation_rename(endpoint_id, oldpath="/~/tutorial_dir",
                    newpath="/~/tutorial_dir_renamed")
print (r.data["message"])

## Deactivating Endpoints

After using an endpoint you can deactivate it so that it cannot be used until it is reactivated. 

In [ ]:
r = tc.endpoint_deactivate(tutorial_endpoint_1)
print (r.data["message"])

# Task submission and management

The Globus task interface allows you to create and manage asynchronous transfer tasks. 

## Transfer

Creating a transfer is a two stage process. First you must create a description of the data you want to transfer and then you can submit the request to Globus to transfer that data. 


In [ ]:
# help(tc.submit_transfer)
source_endpoint_id = tutorial_endpoint_1
source_path = '/share/godata/'

dest_endpoint_id = tutorial_endpoint_2
dest_path = '/~/'

label = "My tutorial transfer"

transfer_items = []

## Recursively transfer source path contents
transfer_items.append(
    tc.make_submit_transfer_item(source_path,
                                 dest_path,
                                 recursive=True))
## Alternatively, transfer a specific file
# transfer_items.append(
#     tc.make_submit_transfer_item("/source/path/file.txt",
#                                  "/dest/path/file.txt"))

# Ensure endpoints are activated
ep1 = tc.endpoint_autoactivate(tutorial_endpoint_1)
ep2 = tc.endpoint_autoactivate(tutorial_endpoint_2)

transfer_data = tc.make_submit_transfer_data(
    source_endpoint_id,
    dest_endpoint_id,
    transfer_items, label=label)
transfer_result = tc.submit_transfer(transfer_data)
task_id = transfer_result.data["task_id"]
print("Task ID: %s" % task_id)

## Get Task By ID

While the task is running, or after completion, you can get information that describes the transfer task. 

In [ ]:
r = tc.get_task(transfer_result.data['task_id'])
print ("Label: %s" % r.data["label"])
print ("Status: %s" % r.data["status"])
print ("Transfer: %s -> %s" %(r.data["source_endpoint_display_name"], r.data["destination_endpoint_display_name"]))
    
if r.data["status"] == "SUCCEEDED":
    print ("Bytes transferred: %s" % r.data["bytes_transferred"])
    print ("Files transferred: %s" % r.data["files_transferred"])
    print ("Transfer rate: %s bps" % r.data["effective_bytes_per_second"])

## Check destination endpoint

After the transfer has finished you can list the contents of the destination endpoint

In [ ]:
r = tc.operation_ls(dest_endpoint_id,path=dest_path)
print("==== Endpoint_ls for endpoint %s %s ===="%(dest_endpoint_id,dest_path))
for item in r.data['DATA']:
    print("%s: %s [%s]"%(item['type'],item['name'],item['size']))

## Get task list

You can get a list of past or current tasks with the following call. Note, you can also use a variety of filters to control the scope of results. 

In [ ]:
# help(tc.task_list)
r = tc.task_list(num_results=10)
for i,item in enumerate(r):
    print(item.data['status'],
          item.data['task_id'], 
          item.data['type'],
          item.data['source_endpoint_display_name'],
          item.data['destination_endpoint_display_name'],
          item.data['label'])

## Filter task list

Retrieve only active tasks.  TODO: Link to filters. 

In [ ]:
r = tc.task_list(num_results=10, filter="status:ACTIVE")
for i,item in enumerate(r):
    print(item.data['status'],
          item.data['task_id'], 
          item.data['type'],
          item.data['source_endpoint_display_name'],
          item.data['destination_endpoint_display_name'],
          item.data['label'])

## Update task by id 

You can make changes to a task while is is still running. For exmaple, updating the label or deadline. 

In [ ]:
#help(tc.update_task)
task_doc = {
        'DATA_TYPE': 'task',
        'label' : 'updated tutorial label'
}

tc.update_task(task_id, task_doc)

r = tc.get_task(task_id)
print ("Label: %s" % r.data["label"])


## Cancel task

You can also cancel a running task. 

In [ ]:
#help(tc.cancel_task)
r = tc.cancel_task(task_id)
print (r.data["message"])

## Get event list for task

Every task stores periodic event markers (e.g., errors, performance markers, etc.) You can retrieve and filter this list as follows. 

In [ ]:
#help(tc.task_event_list)
r = tc.task_event_list(task_id, num_results=10)
for e in r:
    print (e.data['time'], e.data['code'], e.data['is_error'], e.data['details'])


## Get task pause info

In [ ]:
#help(tc.task_pause_info)
r = tc.task_pause_info(task_id)
print (r)

# Bookmarks

Bookmarks allow you to keep a list of frequently used endpoints and paths. Full management capabilities (create, retrieve, update, delete) are supported on bookmarks. 

## Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
endpoint_id = tutorial_endpoint_1
endpoint_path = '/share/godata/'
r = tc.create_bookmark({"endpoint_id":endpoint_id, "path":endpoint_path,"name":bookmark_name})
bookmark_id = r.data['id']
print(r)

## Get a list of bookmarks

In [ ]:
r = tc.bookmark_list()
for b in r:
    print (b.data['name'], b.data['path'], b.data['id'])

## Update a bookmark


In [ ]:
bookmark_data = {
    'name': 'My Updated Tutorial Bookmark'
}
r = tc.update_bookmark(bookmark_id, bookmark_data)
print (r)

## Delete a Bookmark

In [ ]:
r = tc.delete_bookmark(bookmark_id)
print (r)

# Shared endpoints

## Create a shared endpoint

In [ ]:
# create a dir to share
endpoint_id = tutorial_endpoint_1
endpoint_path = '/~/shared_dir2'
r = tc.operation_mkdir(endpoint_id,path=endpoint_path)

# define the shared endpoint 
shared_ep = {"DATA_TYPE":"endpoint",
             "host_endpoint":  tutorial_endpoint_1,
             "host_path":endpoint_path,
             "display_name":"My Tutorial Shared Endpoint2",
             "description":"",
             "keywords":"",
             "contact_email":None,
             "info_link":None,
             "force_encryption":False}

r = tc.post("shared_endpoint", shared_ep)
shared_endpoint_id = r.data['id']
print (r)


## Get endpoint information

In [ ]:
r = tc.get_endpoint(shared_endpoint_id)
print("Display name: %s" % r.data["display_name"])
print("Owner: %s" % r.data["owner_string"])
print("Host Endpoint ID: %s" % r.data["host_endpoint_id"])

## Get a list of shared endpoints

In [ ]:
endpoints = tc.endpoint_search(filter_scope="shared-by-me")
print("==== Displaying shared endpoints ===")
for r in endpoints:
    print("%s (%s)"%(r.data["display_name"], r.data["id"]))


## Access_manager role

## Add a new access control rule

You can share access to different paths within your shared endpoint with users, groups, or publically. Here is an example of sharing with the tutorial users group. 

In [ ]:
rule_data = {
    'DATA_TYPE': 'access',
    'permissions': 'rw',
    'principal' : tutorial_users_group,
    'principal_type' : 'group',
    #'principal': 'IDENTITY_ID',
    #'principal_type': 'identity',
    'path': '/'
}

r= tc.add_endpoint_acl_rule(shared_endpoint_id, rule_data)
access_rule_id = r.data['access_id']
print (r)

## Get list off access rules

In [ ]:
r = tc.endpoint_acl_list(shared_endpoint_id)
for a in r:
    print (a.data['id'], a.data['principal_type'], a.data['principal'], a.data['permissions'], a.data['path'])

## Get access rule by id

In [ ]:
r = tc.get_endpoint_acl_rule(shared_endpoint_id, access_rule_id)
print (r)


## Update access rule

In [ ]:
rule_data = {
    'DATA_TYPE': 'access',
    'permissions': 'r',
    'principal' : tutorial_users_group,
    'principal_type' : 'group',
    'path': '/'
}
r = tc.update_endpoint_acl_rule(shared_endpoint_id, access_rule_id, rule_data)
print (r)

## Delete access rule

In [ ]:
r = tc.delete_endpoint_acl_rule(shared_endpoint_id, access_rule_id)
print (r)